# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df = pd.read_csv("output_data/cities.csv")
df.head()


,city,latitude,longitude,temperature,feels_like,humidity,pressure,wind_speed,cloudiness
0,narsaq,60.92,-46.05,30.20,24.53,86,975,2.44,100
1,mataura,-46.19,168.86,55.99,50.72,71,1022,8.01,95
2,bluff,-46.60,168.33,55.99,50.72,71,1022,8.01,77
3,rikitea,-23.12,-134.97,73.09,69.93,78,1017,15.66,89
4,ushuaia,-54.80,-68.30,41.00,19.02,65,1008,32.21,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         583 non-null    object 
 1   latitude     583 non-null    float64
 2   longitude    583 non-null    float64
 3   temperature  583 non-null    float64
 4   feels_like   583 non-null    float64
 5   humidity     583 non-null    int64  
 6   pressure     583 non-null    int64  
 7   wind_speed   583 non-null    float64
 8   cloudiness   583 non-null    int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 41.1+ KB


In [5]:
# set locations to lat and long
locations = df[["latitude", "longitude"]]

locations

,latitude,longitude
0,60.92,-46.05
1,-46.19,168.86
2,-46.60,168.33
3,-23.12,-134.97
4,-54.80,-68.30
...,...,...
578,58.30,-134.42
579,60.37,93.04
580,-34.48,-54.33
581,7.77,-72.22


In [10]:
# set humidity
humidity = df["humidity"]

humidity

0      86
1      71
2      71
3      78
4      65
       ..
578    88
579    92
580    94
581    91
582    84
Name: humidity, Length: 583, dtype: int64

In [26]:
# special thanks to https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
# figure customization
figure_layout = {
    'width': '1000px',
    'height': '650px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# create the heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=500)

heatmap_layer.gradient = [
    'gray',
    'yellow',
    'red'
]

heatmap_layer.opacity = 0.5

# add heatmap layer
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='650px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         583 non-null    object 
 1   latitude     583 non-null    float64
 2   longitude    583 non-null    float64
 3   temperature  583 non-null    float64
 4   feels_like   583 non-null    float64
 5   humidity     583 non-null    int64  
 6   pressure     583 non-null    int64  
 7   wind_speed   583 non-null    float64
 8   cloudiness   583 non-null    int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 41.1+ KB


In [32]:
# create a mask of my ideal weather
mask = (df.feels_like < 75) & (df.feels_like > 71) & (df.wind_speed < 10) & (df.humidity < 50)
ideal_df = df.loc[mask].reset_index(drop=True)

# how many cities?
print(len(ideal_df))

ideal_df.head()

10


,city,latitude,longitude,temperature,feels_like,humidity,pressure,wind_speed,cloudiness
0,new norfolk,-42.78,147.06,76.14,72.36,28,1021,3.00,37
1,victoria,22.29,114.16,75.54,74.70,45,1017,3.00,83
2,manyana,-23.40,21.72,77.77,74.77,38,1012,5.53,98
3,atar,20.52,-13.05,81.64,74.77,16,1014,5.61,87
4,sibi,29.54,67.88,80.04,74.43,16,1020,3.06,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
# create open lists to store data found
hotelname_list = []
hoteladdress_list = []

# find the closest hotel from each city's coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in ideal_df.iterrows():
    
    # build f string to match lat long syntax
    location = f"{row.latitude}, {row.longitude}"
    # add location parameter
    params['location'] = location
    
    # make API request
    response = requests.get(base_url, params=params).json()
    
    # get results
    results = response['results']
    
    try:
        # get data
        name = results[0]['name']
        address = results[0]['vicinity']
        
        # print for processing hotel log
        print(f'Got data for {row.city}; closest hotel is {name}; city index {index+1} of {len(ideal_df)}')
        
        # add data
        hotelname_list.append(name)
        hoteladdress_list.append(address)
              
    # to inject a result if there is an error    
    except (KeyError, IndexError):
        print("There does not appear to be a nearby hotel.")
        hotelname_list.append(np.nan)
        hoteladdress_list.append(np.nan)

            

Got data for new norfolk; closest hotel is New Norfolk District Hospital; city index 1 of 10
Got data for victoria; closest hotel is Sheung Wan/ Central Cozy Studio B 30sec to MTR; city index 2 of 10
There does not appear to be a nearby hotel.
Got data for atar; closest hotel is STUDIO ADRAR MEDIA Pour Filmer Et Monter Les Films Ceremonies BOUHE HOUSSEIN; city index 4 of 10
Got data for sibi; closest hotel is Gorgaje Muhalla; city index 5 of 10
Got data for leshan; closest hotel is 乐山市档案馆; city index 6 of 10
Got data for kang; closest hotel is Goo Baitshepi Inn; city index 7 of 10
Got data for alwar; closest hotel is Daauji Temple; city index 8 of 10
Got data for wer; closest hotel is Bablu; city index 9 of 10
Got data for chenghai; closest hotel is France Gongji; city index 10 of 10


In [52]:
# populate with all the lists!
ideal_df['hotel'] = hotelname_list
ideal_df['hotel_address'] = hoteladdress_list

ideal_df.head()

,city,latitude,longitude,temperature,feels_like,humidity,pressure,wind_speed,cloudiness,hotel,hotel_address
0,new norfolk,-42.78,147.06,76.14,72.36,28,1021,3.00,37,New Norfolk District Hospital,New Norfolk
1,victoria,22.29,114.16,75.54,74.70,45,1017,3.00,83,Sheung Wan/ Central Cozy Studio B 30sec to MTR,Sheung Wan
2,manyana,-23.40,21.72,77.77,74.77,38,1012,5.53,98,NaN,NaN
3,atar,20.52,-13.05,81.64,74.77,16,1014,5.61,87,STUDIO ADRAR MEDIA Pour Filmer Et Monter Les F...,"N1, Atar"
4,sibi,29.54,67.88,80.04,74.43,16,1020,3.06,0,Gorgaje Muhalla,"near WAPDA Colony Gharib Abad, Sibi"


In [54]:
# conversions
hotel_df = ideal_df
hotel_df['Hotel Name'] = hotel_df.hotel
hotel_df['City'] = hotel_df.city
hotel_df['Lat'] = hotel_df.latitude
hotel_df['Lng'] = hotel_df.longitude

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# NOTE: Do not change any of the code in this cell 
# ***could not find where the country info was stored so copied and deleted the country tag***

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]

In [58]:
coordinates = df[["latitude", "longitude"]]

# special thanks to https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
# figure customization

figure_layout = {
    'width': '1000px',
    'height': '650px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# create the heatmap layer
heatmap_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating=False, max_intensity=500)

heatmap_layer.gradient = [
    'gray',
    'yellow',
    'red'
]

heatmap_layer.opacity = 0.5

# add heatmap layer
fig.add_layer(heatmap_layer)



# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='650px', padding='1px', width='1000px'))